In [1]:
import pandas as pd
import math
import fict_api4 as environment
import numpy

a = [0] * 1000
a[40] = "Netherlands"
a[616] = "Poland"
a[36] = "Austria"
a[276] = "Germany"
a[56] = "Belgium"

event_cnt = 10
hour = pd.to_timedelta("0:1:0")

class item:
    cnt = 0
    rating = 0

class node:
    
    def __init__(self, arrive, depart, cash, counry_id):
        self.arrivial_time = arrive
        self.departure_time = depart
        self.cash = cash
        self.country_id = country_id
        
def radian(x):
    return (x * math.pi) / 180

def distance_from_points(f1, f2, q1, q2):
    f1 = radian(f1)
    f2 = radian(f2)
    q1 = radian(q1)
    q2 = radian(q2)
    temp1 = ((math.sin(f2 - f1) / 2)**2) + math.cos(f1) * math.cos(f2) * ((math.sin(q2 - q1) / 2)**2)
    return 2 * 6371 * math.asin(temp1**0.5)

def distance_from_areas(i1, i2):
    f1 = table.loc[i1].x
    f2 = table.loc[i2].x
    q1 = table.loc[i1].y
    q2 = table.loc[i2].y
    return distance_from_points(f1, f2, q1, q2)

def get_time_taxi(dist):
    return dist / 40.0

def get_time_fit(dist):
    return dist / 7

def taxi_count(dist):
    return (15 * 70 * dist) / 5

def id_to_sights(id):
    res = a[id]
    if (res == "Netherlands"):
        t = pd.read_json('Netherlands')
    if (res == "Poland"):
        t = pd.read_json('Poland')
    if (res == "Belgium"):
        t = pd.read_json('Belgium')
    if (res == "Austria"):
        t = pd.read_json('Austria')
    if (res == "Germany"):
        t = pd.read_json('Germany')
    t.time = pd.to_timedelta(t.time)
    return t

In [7]:
def get_random_ans(table, event_cnt):
    cur_rating = 0
    cur_cnt = 0
    cur_cash = 0
    cur_time = 0
    ans = []
    ids = list(range(event_cnt))
    weights = list(table.rating)
#     print(len(ids), len(weights))
    weights = [x / sum(weights) for x in weights]
    while(len(ids)):
        temp = numpy.random.choice(range(len(ids)), 1, p=weights)[0]
#         print(type(temp), temp)
        ans.append(ids[temp])
        ids.pop(temp)
        weights.pop(temp)
        weights = [x / sum(weights) for x in weights]
    return ans

In [68]:
def get_ans(table, event_cnt, max_cash, start_x, start_y):
    print(start_x, start_y)
    print(table)
    print(event_cnt)
    cnt_1 = 0
    cnt_2 = 0
    hour = pd.to_timedelta("0:1:0")
    ans_rating = 0
    ans_cnt = 0
    ans_cash = 0
    ans = []
    ans_time = pd.to_timedelta("0:0:0")
    for _ in range(150):
        cur_rating = 0
        cur_cnt = 0
        cur_cash = 0
        cur_time = pd.to_timedelta("9:0:0")
        new_ans = get_random_ans(table, event_cnt)
        prev = -1
        for j,i in enumerate(new_ans):
            dist = 0
            if prev == -1:
                dist = distance_from_points(start_x, table.loc[i].x, start_y, table.loc[i].y)
            else:
                dist = distance_from_areas(prev, i)
            cur_cnt += 1
            cur_cash += table.loc[i].cost  
            cur_time += table.loc[i].time
            cur_rating += table.loc[i].rating
            if dist <= 0.8:
                needTime = get_time_fit(dist)
                cur_time += needTime * hour
            else:
                needTime = get_time_taxi(dist)
                cur_time += needTime * hour
                cur_cash += taxi_count(dist)
            if (cur_cash <= max_cash) and (cur_time < pd.to_timedelta("23:59:59")):
                if (cur_rating > ans_rating):
                    ans = new_ans[:j + 1]
                    ans_rating = cur_rating
                    ans_cash = cur_cash
                    ans_time = cur_time
                    ans_cnt = cur_cnt
            prev = i
    return (ans_cash, ans_time, ans)
            
    
def get_hotel(env, table):
    curK = 1e9
    num = 0
    for i in range(len(env)):
        curSum = 0
        for j in range (len(table)):
            curSum += distance_from_points(env.iloc[i].x, table.loc[j].x, env.iloc[i].y, table.loc[j].y)
        if (curSum < curK):
            curK = curSum
            num = i;
    print(curSum)
    return num

In [64]:
table

,name,x,y,cost,time,rating
0,Brandenburg Gate,52.516278,13.375515,0,00:15:00,10
1,Reichstag,52.518620,13.376187,0,01:00:00,10
2,East Side Gallery,52.505408,13.436360,0,00:15:00,6
3,Museum Island,52.516890,13.397764,1400,03:00:00,7
4,Unter den Linden,52.516848,13.387012,0,00:20:00,5
5,Kaiser Wilhelm Memorial Church,52.504821,13.332888,700,01:00:00,4
6,Postdam Platz,52.509652,13.373750,0,00:30:00,4
7,New Museum,52.520138,13.395452,1400,02:00:00,8
8,Checkpoint Charlie,52.507591,13.388181,0,00:15:00,7
9,Berlin Flea Market,52.513591,13.331188,-1,00:45:00,3


In [86]:
start_time = pd.to_datetime("2/8/2020") 
end_time = pd.to_datetime("2/9/2020")
id_ = 276
cash = 20000
country = a[id_]
table = id_to_sights(id_)
env_ = environment.Environment()
env = env_.get_hotels(276, start_time, end_time)
temp = get_hotel(env, table)
start_x = env.iloc[temp].x
start_y = env.iloc[temp].y
cur_time = pd.to_timedelta("5:30:0")
k = get_ans(table, event_cnt, 20000, start_x, start_y)
ans = []
prev = -1
ans.append({'type':'flight', 'name':'flight to' + country, 'time_in':(str(cur_time)), 'time_out':(str(cur_time + pd.to_timedelta("3:0:0")))})
cur_time = "9:0:0"
ans.append({'type':'hotel', 'name':'check in' + env.iloc[temp]['name'], 'time_in':(str(cur_time)), 'time_out':(str(cur_time + pd.to_timedelta("1:0:0")))})
for i in k[2]:
    dist = 0
    time = "0:0:0"
    if prev == -1:
        dist = 1
    else:
        dist = distance_from_areas(i, prev)
    if dist <= 0.8:
        cur_time += get_time_fit(dist) * hour
    else:
        cur_time += get_time_taxi(dist) * hour
    prev = i
    ans.append({'type':'sightseeing', 'name': table.loc[i].name, 'time_in':(str(cur_time)), 'time_out':(str(cur_time + table.loc[i].time))})
    cur_time += table.loc[i].time 
print(ans)

Generating time_to_planes dataset...
Generating country_to_hotels dataset...
Done
169.53589379306504
52.50542250193426 13.556833535119894
                             name          x          y  cost     time  rating
0                Brandenburg Gate  52.516278  13.375515     0 00:15:00      10
1                       Reichstag  52.518620  13.376187     0 01:00:00      10
2               East Side Gallery  52.505408  13.436360     0 00:15:00       6
3                   Museum Island  52.516890  13.397764  1400 03:00:00       7
4                Unter den Linden  52.516848  13.387012     0 00:20:00       5
5  Kaiser Wilhelm Memorial Church  52.504821  13.332888   700 01:00:00       4
6                   Postdam Platz  52.509652  13.373750     0 00:30:00       4
7                      New Museum  52.520138  13.395452  1400 02:00:00       8
8              Checkpoint Charlie  52.507591  13.388181     0 00:15:00       7
9              Berlin Flea Market  52.513591  13.331188    -1 00:45:00  

In [87]:
whole_pass = dict()
whole_pass['name'] = country
whole_pass['dates'] = str(start_time) + ' - ' + str(start_time + cur_time)
whole_pass['cost'] = str(k[0])

In [88]:
[whole_pass]

[{'name': 'Germany',
  'dates': '2020-02-08 00:00:00 - 2020-02-08 18:20:32.031939803',
  'cost': '8612.770988930271'}]